In [3]:
import numpy as np
import pandas as pd
import pydicom
%matplotlib inline
import matplotlib.pyplot as plt
import keras 

from skimage.transform import resize
import json

In [4]:
# This function reads in a .dcm file, checks the important fields for our device, and returns a numpy array
# of just the imaging data
def check_dicom(filename): 
    # todo
    print(f"Load file {filename}")

    dcm = pydicom.dcmread(filename)    
    
    # print(dcm)
    
    headers = {
        "PatientID":dcm.PatientID,
        "PatientAge": dcm.PatientAge,
        "PatientSex": dcm.PatientSex,
        "ImagingModality": dcm.Modality,
        "PatientPosition": dcm.PatientPosition,
        "BodyPartExamined": dcm.BodyPartExamined,
        "TypeOfFinding": dcm.StudyDescription,
        "RowNum": dcm.Rows,
        "ColNum": dcm.Columns,
    }
    
    modality = headers["ImagingModality"]
    if modality != "DX":
        print(f"WARNING: Image modality should be 'Digital Radiography' (DX): {modality}")
        
    body = headers["BodyPartExamined"]    
    if body != "CHEST":
        print(f"WARNING: Body Part Examined is not CHEST: {body}")
    
    position = headers["PatientPosition"] 
    if position not in ["PA", "AP"]:
        print(f"WARNING: The patient position should be 'PA' or 'AP' : {position}")
                
    img = dcm.pixel_array
    
    return img
    
    
# This function takes the numpy array output by check_dicom and 
# runs the appropriate pre-processing needed for our model input
def preprocess_image(img,img_mean,img_std,img_size): 

    # todo
    img_mean = img.mean()
    proc_img=  resize((img - img_mean) / img_std, img_size)
    
    return proc_img

# This function loads in our trained model w/ weights and compiles it 
def load_model(model_path, weight_path):
    
    with open(model_path, "r") as fp:
        loaded_model_json = fp.read() 
        
    model = keras.models.model_from_json(loaded_model_json)
    model.load_weights(weight_path)
    
    return model

# This function uses our device's threshold parameters to predict whether or not
# the image shows the presence of pneumonia using our trained model
def predict_image(model, img, thresh): 
    res = model.predict(img)
    prediction = 1 if res > thresh else 0

    return prediction 

In [5]:
check_dicom('test1.dcm')

Load file test1.dcm


array([[199, 175, 152, ..., 252, 251, 251],
       [150, 128, 114, ..., 243, 250, 252],
       [122, 110, 102, ..., 193, 222, 239],
       ...,
       [ 44,  47,  51, ...,  16,  16,  13],
       [ 44,  48,  52, ...,  16,  16,  14],
       [ 45,  48,  53, ...,  17,  17,  15]], dtype=uint8)

In [6]:
test_dicoms = ['test1.dcm','test2.dcm','test3.dcm','test4.dcm','test5.dcm','test6.dcm']

model_path = "my_model.json"
weight_path = "xray_class_my_model.best.hdf5"

IMG_SIZE=(1,224,224,3) # This might be different if you did not use vgg16
img_mean = 0
img_std = 255.0

my_model = load_model(model_path, weight_path)
thresh = 0.77

# use the .dcm files to test your prediction
for i in test_dicoms:
    
    img = np.array([])
    img = check_dicom(i)
    
    if img is None:
        continue
        
    img_proc = preprocess_image(img,img_mean,img_std,IMG_SIZE)
    pred = predict_image(my_model,img_proc,thresh)
    print(pred)

Load file test1.dcm
0
Load file test2.dcm
0
Load file test3.dcm
0
Load file test4.dcm
0
Load file test5.dcm
0
Load file test6.dcm
0
